In [13]:
import pandas as pd
import altair as alt
import plotly.express as px
import pyarrow

alt.data_transformers.enable("vegafusion")

DataTransformerRegistry.enable('vegafusion')

In [14]:
PAYMENT_DATA_PATH = "../pipeline/data/payment_data.csv"
TRUCK_DATA_PATH = "../pipeline/data/truck_data.csv"
TRANSACTION_DATA_PATH = "../pipeline/data/transaction_data.csv"

In [15]:
payment_data_df = pd.read_csv(PAYMENT_DATA_PATH)
truck_data_df = pd.read_csv(TRUCK_DATA_PATH)
transaction_data_df = pd.read_csv(TRANSACTION_DATA_PATH)

In [16]:
# SELECT
# 	transaction_id, dt.truck_name, dpm.payment_method, total, "at", dt.has_card_reader, fsa_rating
# FROM FACT_Transaction AS ft
# JOIN DIM_Payment_Method AS dpm
# 	ON ft.payment_method_id = dpm.payment_method_id 
# JOIN DIM_Truck AS dt
# 	ON ft.truck_id = dt.truck_id

main_df = transaction_data_df.merge(payment_data_df, on="payment_method_id") \
    .merge(truck_data_df, on="truck_id")

main_df = main_df[["transaction_id", "truck_name", "payment_method", "total", "at", "has_card_reader", "fsa_rating"]]

main_df

,transaction_id,truck_name,payment_method,total,at,has_card_reader,fsa_rating
0,1,Burrito Madness,cash,960.0,2025-10-19 09:04:00,1,4
1,2,Burrito Madness,cash,700.0,2025-10-19 09:22:00,1,4
2,3,Burrito Madness,card,700.0,2025-10-19 09:29:00,1,4
3,4,Burrito Madness,cash,930.0,2025-10-19 09:42:00,1,4
4,5,Burrito Madness,card,700.0,2025-10-19 10:11:00,1,4
...,...,...,...,...,...,...,...
5612,5613,SuperSmoothie,cash,599.0,2025-10-25 16:23:00,0,3
5613,5614,SuperSmoothie,cash,499.0,2025-10-25 16:33:00,0,3
5614,5615,SuperSmoothie,cash,599.0,2025-10-25 16:39:00,0,3
5615,5616,SuperSmoothie,cash,599.0,2025-10-25 16:47:00,0,3


In [17]:
# Convert 'at' to just the date
main_df['date'] = pd.to_datetime(main_df['at']).dt.date

# Group by truck and date, sum the totals
daily_totals = main_df.groupby(['truck_name', 'date'])['total'].sum().reset_index()

# Calculate cumulative sum for each truck
# Sort by date first, then use groupby with cumsum()
daily_totals = daily_totals.sort_values('date')
daily_totals['cumulative_total_pennies'] = daily_totals.groupby('truck_name')[
    'total'].cumsum()
daily_totals['cumulative_total_pounds'] = daily_totals['cumulative_total_pennies'] / 100.0

daily_totals

,truck_name,date,total,cumulative_total_pennies,cumulative_total_pounds
0,Burrito Madness,2025-10-19,256819.0,256819.0,2568.19
28,SuperSmoothie,2025-10-19,90447.0,90447.0,904.47
21,Kings of Kebabs,2025-10-19,95983.0,95983.0,959.83
14,Hartmann's Jellied Eels,2025-10-19,9465.0,9465.0,94.65
7,Cupcakes by Michelle,2025-10-19,60402.0,60402.0,604.02
35,Yoghurt Heaven,2025-10-19,154956.0,154956.0,1549.56
15,Hartmann's Jellied Eels,2025-10-20,14052.0,23517.0,235.17
22,Kings of Kebabs,2025-10-20,72597.0,168580.0,1685.80
8,Cupcakes by Michelle,2025-10-20,178208.0,238610.0,2386.10
1,Burrito Madness,2025-10-20,115573.0,372392.0,3723.92


In [18]:
line = alt.Chart(daily_totals).mark_line().encode(
    x=alt.X('date:T', title='Date'),
    y=alt.Y('cumulative_total_pounds:Q', title='Cumulative Total (£)'),
    color=alt.Color('truck_name:N', title='Truck Name')
).properties(
    title='Total Sales Over Time by Truck'
)

line

alt.Chart(...)

In [19]:
# Bar chart showing payment method distribution for each truck

payment_distribution_df = main_df.groupby(['truck_name', 'payment_method'])[
    'transaction_id'].count().reset_index()
payment_distribution_df = payment_distribution_df.rename(
    columns={'transaction_id': 'count'})

payment_method_df = payment_distribution_df
payment_method_chart = px.bar(payment_method_df,
                                title="Distribution of Payment Methods by Truck",
                                x='truck_name',
                                y='count',
                                color='payment_method',
                                labels={'truck_name': 'Truck Name',
                                        'count': 'Number of Transactions',
                                        'payment_method': 'Payment Method'
                                        }
                                )

payment_method_chart

ValueError: Mime type rendering requires nbformat>=4.2.0 but it is not installed

Figure({
    'data': [{'hovertemplate': ('Payment Method=card<br>Truck N' ... 'ansactions=%{y}<extra></extra>'),
              'legendgroup': 'card',
              'marker': {'color': '#636efa', 'pattern': {'shape': ''}},
              'name': 'card',
              'orientation': 'v',
              'showlegend': True,
              'textposition': 'auto',
              'type': 'bar',
              'x': array(['Burrito Madness', 'Cupcakes by Michelle', "Hartmann's Jellied Eels",
                          'Kings of Kebabs', 'Yoghurt Heaven'], dtype=object),
              'xaxis': 'x',
              'y': {'bdata': 'iQKWAq4AAQJdAg==', 'dtype': 'i2'},
              'yaxis': 'y'},
             {'hovertemplate': ('Payment Method=cash<br>Truck N' ... 'ansactions=%{y}<extra></extra>'),
              'legendgroup': 'cash',
              'marker': {'color': '#EF553B', 'pattern': {'shape': ''}},
              'name': 'cash',
              'orientation': 'v',
              'showlegend': True,
              'textposition': 'auto',
              'type': 'bar',
              'x': array(['Burrito Madness', 'Cupcakes by Michelle', "Hartmann's Jellied Eels",
                          'Kings of Kebabs', 'SuperSmoothie', 'Yoghurt Heaven'], dtype=object),
              'xaxis': 'x',
              'y': {'bdata': 'UQIfAn4AewEMA1EC', 'dtype': 'i2'},
              'yaxis': 'y'}],
    'layout': {'barmode': 'relative',
               'legend': {'title': {'text': 'Payment Method'}, 'tracegroupgap': 0},
               'template': '...',
               'title': {'text': 'Distribution of Payment Methods by Truck'},
               'xaxis': {'anchor': 'y', 'domain': [0.0, 1.0], 'title': {'text': 'Truck Name'}},
               'yaxis': {'anchor': 'x', 'domain': [0.0, 1.0], 'title': {'text': 'Number of Transactions'}}}
})